##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# A simple classification model using Keras with Cloud TPUs


## Overview

This notebook shows how to use Keras to build a simple classification model. The model can train, evaluate, and generate predictions using Cloud TPUs. It uses the iris dataset to predict the species of the flower and also shows how to use your own data instead of using pre-loaded data. This model uses 4 input features ***(SepalLength, SepalWidth, PetalLength, PetalWidth)*** to determine one of these flower species ***(Setosa, Versicolor, Virginica)***.

The model trains for 50 epochs and completes in approximately 2 minutes.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.


**NOTE:** This tutorial is designed to show how to write a simple model using Keras. It should not be used for comparision with training on CPU's because of the very small amount of data being used.

## Learning objectives

In this Colab, you will learn how to:
*   Define a Keras model with 2 hidden layers and 10 nodes in each layer.
*   Create and compile a Keras model on TPU with a distribution strategy.
*   Train, evaluate, and and generate predictions on Cloud TPU.






## Instructions

<h3>  &nbsp;&nbsp;Train on TPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a></h3>

   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All**. You can also run the cells manually with Shift-ENTER. 

## Data, model, and training

### Imports

In [0]:
import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras


In [2]:
print(tf.__version__)
import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/classification_iris_data_with_keras.ipynb')

1.14.0


### Resolve TPU Address

In [3]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

with tf.Session(TF_MASTER) as session:
  print ('List of devices:')
  pprint.pprint(session.list_devices())

List of devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 633935839914717665),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3090728273145568667),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 16231206090346540868),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1906202020534649681),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3168951952970533977),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 11881921795421459222),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 12929031801876524727),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 779556975695693),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 7550871174155397328),
 _DeviceAttributes(/job:tpu_work

### FLAGS used as model params

In [0]:
# Model specific parameters

# TPU address
tpu_address = TF_MASTER

# Number of epochs
epochs = 50

# Number of steps_per_epoch
steps_per_epoch = 5

# NOTE: Total number of training steps = Number of epochs * Number of steps_per_epochs

### Download training input data and define prediction input & output

In [0]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

PREDICTION_INPUT_DATA = {
    'SepalLength': [6.9, 5.1, 5.9, 6.0, 5.5, 6.2, 5.5, 6.3],
    'SepalWidth': [3.1, 3.3, 3.0, 3.4, 2.5, 2.9, 4.2, 2.8],
    'PetalLength': [5.4, 1.7, 4.2, 4.5, 4.0, 4.3, 1.4, 5.1],
    'PetalWidth': [2.1, 0.5, 1.5, 1.6, 1.3, 1.3, 0.2, 1.5],
}

PREDICTION_OUTPUT_DATA = ['Virginica', 'Setosa', 'Versicolor', 'Versicolor', 'Versicolor', 'Versicolor', 'Setosa', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

### Define a Keras model (2 hidden layers with 10 neurons in each)

In [0]:
def get_model():
  return keras.Sequential([
    keras.layers.Dense(10, input_shape=(4,), activation=tf.nn.relu, name = "Dense_1"),
    keras.layers.Dense(10, activation=tf.nn.relu, name = "Dense_2"),
    keras.layers.Dense(3, activation=None, name = "logits"),
    keras.layers.Dense(3, activation=tf.nn.softmax, name = "softmax")
  ])

### Compiling the model with a distribution strategy
To make the model usable by a TPU, we first must create and compile it using a distribution strategy.

In [7]:
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TF_MASTER)
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)


W0925 19:52:17.934039 140563505481600 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [8]:
with strategy.scope():
  model = get_model()
  model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1), 
                loss='sparse_categorical_crossentropy',
                metrics=['sparse_categorical_crossentropy'])

model.summary()

W0925 19:52:32.680455 140563505481600 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
Dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
logits (Dense)               (None, 3)                 33        
_________________________________________________________________
softmax (Dense)              (None, 3)                 12        
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


### Train the model on TPU

In [9]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

# Train the model
model.fit(
  train_x.values, train_y.values,
  steps_per_epoch = steps_per_epoch,
  epochs=epochs,
)

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


W0925 19:52:42.782965 140563505481600 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/keras/optimizer_v2/adagrad.py:105: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0925 19:52:43.800348 140563505481600 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/keras/engine/training_distributed.py:411: load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Epoch 1/50
5/5 [==============================] - 0s 99ms/step - loss: 1.2053 - sparse_categorical_crossentropy: 1.2053
Epoch 2/50
5/5 [==============================] - 0s 10ms/step - loss: 0.8204 - sparse_categorical_crossentropy: 0.8204
Epoch 3/50
5/5 [==============================] - 0s 11ms/step - loss: 0.6280 - sparse_categorical_crossentropy: 0.6280
Epoch 4/50
5/5 [==============================] - 0s 9ms/step - loss: 0.5439 - sparse_categorical_crossentropy: 0.5439
Epoch 5/50
5/5 [==============================] - 0s 11ms/step - loss: 0.4785 - sparse_categorical_crossentropy: 0.4785
Epoch 6/50
5/5 [==============================] - 0s 9ms/step - loss: 0.4380 - sparse_categorical_crossentropy: 0.4380
Epoch 7/50
5/5 [==============================] - 0s 9ms/step - loss: 0.4147 - sparse_categorical_crossentropy: 0.4147
Epoch 8/50
5/5 [==============================] - 0s 11ms/step - loss: 0.4183 - sparse_categorical_crossentropy: 0.4183
Epoch 9/50
5/5 [===========================

### Evaluation of the model

In [10]:
model.evaluate(test_x.values, test_y.values,
    batch_size=8)

4/4 [==============================] - 1s 315ms/step


[0.08181199431419373, 0.08726614]

### Save the model

In [0]:
model.save_weights('./DNN_TPU_1024.h5', overwrite=True)

## Prediction

### Prediction data

In [12]:
COLUMNS_NAME=['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
data = pd.DataFrame(PREDICTION_INPUT_DATA, columns=COLUMNS_NAME)
print(data)

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.9         3.1          5.4         2.1
1          5.1         3.3          1.7         0.5
2          5.9         3.0          4.2         1.5
3          6.0         3.4          4.5         1.6
4          5.5         2.5          4.0         1.3
5          6.2         2.9          4.3         1.3
6          5.5         4.2          1.4         0.2
7          6.3         2.8          5.1         1.5


### Prediction on TPU

In [13]:
predictions = model.predict(data.values.astype(np.float32))
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(predictions, PREDICTION_OUTPUT_DATA):
  class_index = np.argmax(pred_dict)
  class_probability = np.max(pred_dict)
  print(template.format(SPECIES[class_index], 100*class_probability, expec))


Prediction is "Virginica" (89.1%), expected "Virginica"

Prediction is "Setosa" (99.7%), expected "Setosa"

Prediction is "Versicolor" (98.2%), expected "Versicolor"

Prediction is "Versicolor" (98.8%), expected "Versicolor"

Prediction is "Versicolor" (94.7%), expected "Versicolor"

Prediction is "Versicolor" (98.9%), expected "Versicolor"

Prediction is "Setosa" (100.0%), expected "Setosa"

Prediction is "Versicolor" (57.3%), expected "Virginica"


### Prediction on CPU

In [0]:
cpu_model = get_model()
cpu_model.load_weights('./DNN_TPU_1024.h5')
cpu_predictions = cpu_model.predict(data)
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(cpu_predictions, PREDICTION_OUTPUT_DATA):
  class_index = np.argmax(pred_dict)
  class_probability = np.max(pred_dict)
  print(template.format(SPECIES[class_index], 100*class_probability, expec))

## What's next

* Learn how to run the same model using [TPUEstimator](https://colab.research.google.com/github/tensorflow/tpu/blob/0e3cfbdfbcf321681c1ac1c387baf7a1a41d8d21/tools/colab/classification_iris_data_with_tpuestimator.ipynb) rather than Keras.
* Learn about [Cloud TPUs](https://cloud.google.com/tpu/docs) that Google designed and optimized specifically to speed up and scale up ML workloads for training and inference and to enable ML engineers and researchers to iterate more quickly.
* Explore the range of [Cloud TPU tutorials and Colabs](https://cloud.google.com/tpu/docs/tutorials) to find other examples that can be used when implementing your ML project.

On Google Cloud Platform, in addition to GPUs and TPUs available on pre-configured [deep learning VMs](https://cloud.google.com/deep-learning-vm/),  you will find [AutoML](https://cloud.google.com/automl/)*(beta)* for training custom models without writing code and [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/) which will allows you to run parallel trainings and hyperparameter tuning of your custom models on powerful distributed hardware.
